In [1]:
import json
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# Load your saved seismic JSON data
with open('../data/seismic_network.json', 'r') as f:
    seismic_data = json.load(f)

# Extract nodes and links
nodes = seismic_data['hybrid_network']['nodes']
links = seismic_data['hybrid_network']['links']
station_index = {node['id']: idx for idx, node in enumerate(nodes)}

# Features and labels
x_list = []
y_list = []

for node in nodes:
    x_list.append([
        node.get("stability_score", 0.0),
        node.get("risk_score", 0.0)
    ])
    y_list.append(1 if node.get("stability") == "unstable" else 0)

x = torch.tensor(x_list, dtype=torch.float)
y = torch.tensor(y_list, dtype=torch.long)

# Edges
edge_list = []
for link in links:
    src = station_index.get(link["source"])
    tgt = station_index.get(link["target"])
    if src is not None and tgt is not None:
        edge_list.append([src, tgt])
        edge_list.append([tgt, src])  # bidirectional

edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

# Create data object
data = Data(x=x, edge_index=edge_index, y=y)


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class MyGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyGCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [4]:
# Correct usage with matching parameters
_x = x.size(1);
model = MyGCN(_x, 16, 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

print(_x)

In [5]:

# Train loop
epochs = 100
losses = []
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    pred = out.argmax(dim=1)
    acc = (pred == data.y).float().mean().item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Accuracy: {acc:.2%}")


Epoch 0 | Loss: 0.6914 | Accuracy: 85.94%
Epoch 10 | Loss: 0.4088 | Accuracy: 90.62%
Epoch 20 | Loss: 0.2757 | Accuracy: 90.62%
Epoch 30 | Loss: 0.2758 | Accuracy: 90.62%
Epoch 40 | Loss: 0.2736 | Accuracy: 90.62%
Epoch 50 | Loss: 0.2654 | Accuracy: 90.62%
Epoch 60 | Loss: 0.2630 | Accuracy: 90.62%
Epoch 70 | Loss: 0.2600 | Accuracy: 90.62%
Epoch 80 | Loss: 0.2567 | Accuracy: 90.62%
Epoch 90 | Loss: 0.2534 | Accuracy: 90.62%


In [ ]:

# Save the trained model
torch.save(model.state_dict(), "gnn_model.pt")
print("✅ Model saved as gnn_model.pt")

✅ Model saved as gnn_model.pt


: 

In [ ]:

# Plot training loss
plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()